This notebook performs the following tasks:
    1. Download .csv file(s) from a AWS S3 Bucket for specific date(s)/hour(s) or a range of dates/hours to a single pandas dataframe
    2. Join ATRCC Data
    3. 

### 1. Install/Load Libraries

In [28]:
pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install pygeohash

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime

import pygeohash as pgh

In [31]:
from sagemaker import get_execution_role
role = get_execution_role()

### 2. Specify Input/Output S3 Buckets

Always, output_bucket = 'partly-cloudy-common-area' or output_bucket = your own bucket

Please do NOT store anything new (i.e., setting output_bucket as) in the following buckets: (1) partly-cloudy-asdb, (2) partly-cloudy-rap-csv, (3) partly-cloudy-rap-parquet and (4) partly-cloudy-common-area/proof-of-concept/

In [32]:
input_bucket = 'partly-cloudy-rap-csv' # <<<<<<<<<<<<<<<<<<<<< Use when reading from master .csv RAP data
#input_bucket = 'partly-cloudy-common-area' # <<<<<<<<<<<<<<<<< Use when reading file(s) from the team common area

output_bucket = 'partly-cloudy-common-area'
subfolder = ''

### 3. For reading specific file(s) from Input S3 Bucket to make a single Dataframe (Run this cell and skip Step 4)

For reading RAP .csv data for a range of dates/times, skip Step 3 and Run Step 4 instead

In [33]:
data_locations = ['s3://partly-cloudy-rap-csv/2021_05_01_13.csv',
                  's3://partly-cloudy-common-area/2021_05_02_20.csv'] # <---------------- Specify

for data_location in data_locations:
    print(data_location)

s3://partly-cloudy-rap-csv/2021_05_01_13.csv
s3://partly-cloudy-common-area/2021_05_02_20.csv


### 4. Reading RAP .csv data for a range of dates/times

In [34]:
def get_dataLocations(bucket_name, firstDT, lastDT):
    if (firstDT >= lastDT):
        lastDT = firstDT

    dateTimes = pd.date_range(firstDT, lastDT, freq= 'H')
    dateTimesSer = pd.Series([str(dateTime) for dateTime in dateTimes], name= 'temp')
    dateTimesDF = pd.DataFrame({'yr':list(dateTimesSer.str.slice(0,4)),
                                'mo':list(dateTimesSer.str.slice(5,7)), 
                                'day':list(dateTimesSer.str.slice(8,10)), 
                                'hr':list(dateTimesSer.str.slice(11,13))})

    dtDF = dateTimesDF.iloc[0:len(dateTimesDF)].copy()
    dat_locs = ['s3://' + bucket_name + '/' + dtDF.iloc[i,0] + "_" + dtDF.iloc[i,1] + "_" + dtDF.iloc[i,2] + "_" + dtDF.iloc[i,3] +'.csv' for i in range(len(dtDF))]
    
    return dat_locs

In [35]:
beginDT = '2021-05-28 00:00:00' # <---------------- Specify (between '2020-06-01 00:00:00' to '2021-05-30 23:00:00')
endDT =   '2021-05-30 23:00:00' # <-----------------Specify (between '2020-06-01 00:00:00' to '2021-05-30 23:00:00')

data_locations = get_dataLocations(input_bucket, beginDT, endDT)

for data_location in data_locations:
    print(data_location)

s3://partly-cloudy-rap-csv/2021_05_28_00.csv
s3://partly-cloudy-rap-csv/2021_05_28_01.csv
s3://partly-cloudy-rap-csv/2021_05_28_02.csv
s3://partly-cloudy-rap-csv/2021_05_28_03.csv
s3://partly-cloudy-rap-csv/2021_05_28_04.csv
s3://partly-cloudy-rap-csv/2021_05_28_05.csv
s3://partly-cloudy-rap-csv/2021_05_28_06.csv
s3://partly-cloudy-rap-csv/2021_05_28_07.csv
s3://partly-cloudy-rap-csv/2021_05_28_08.csv
s3://partly-cloudy-rap-csv/2021_05_28_09.csv
s3://partly-cloudy-rap-csv/2021_05_28_10.csv
s3://partly-cloudy-rap-csv/2021_05_28_11.csv
s3://partly-cloudy-rap-csv/2021_05_28_12.csv
s3://partly-cloudy-rap-csv/2021_05_28_13.csv
s3://partly-cloudy-rap-csv/2021_05_28_14.csv
s3://partly-cloudy-rap-csv/2021_05_28_15.csv
s3://partly-cloudy-rap-csv/2021_05_28_16.csv
s3://partly-cloudy-rap-csv/2021_05_28_17.csv
s3://partly-cloudy-rap-csv/2021_05_28_18.csv
s3://partly-cloudy-rap-csv/2021_05_28_19.csv
s3://partly-cloudy-rap-csv/2021_05_28_20.csv
s3://partly-cloudy-rap-csv/2021_05_28_21.csv
s3://partl

### 5. Ingest Selected Files into a Single Dataframe
The section works for both Sections 3 or 4.

In [36]:
%%time
df = wr.s3.read_csv(path= data_locations, parse_dates= ['dateTime'])

CPU times: user 2min 46s, sys: 15.7 s, total: 3min 2s
Wall time: 1min 1s


In [37]:
df

,dateTime,hPa,FLevel,Nx,Ny,Lat,Lon,Temperature,RH_ice,IsISSR
0,2021-05-28 00:00:00,150,440,1,1,16.281000,-126.138000,204.227,98.33,0
1,2021-05-28 00:00:00,150,440,2,1,16.322011,-125.954684,204.227,98.09,0
2,2021-05-28 00:00:00,150,440,3,1,16.362789,-125.771252,204.227,97.18,0
3,2021-05-28 00:00:00,150,440,4,1,16.403332,-125.587705,204.290,94.41,0
4,2021-05-28 00:00:00,150,440,5,1,16.443642,-125.404045,204.290,92.37,0
...,...,...,...,...,...,...,...,...,...,...
63390595,2021-05-30 23:00:00,450,210,297,225,55.648911,-58.431595,250.439,52.73,0
63390596,2021-05-30 23:00:00,450,210,298,225,55.607604,-58.167947,250.314,53.57,0
63390597,2021-05-30 23:00:00,450,210,299,225,55.565986,-57.904583,250.189,54.42,0
63390598,2021-05-30 23:00:00,450,210,300,225,55.524058,-57.641507,250.126,55.39,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63390600 entries, 0 to 63390599
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   dateTime     datetime64[ns]
 1   hPa          int64         
 2   FLevel       int64         
 3   Nx           int64         
 4   Ny           int64         
 5   Lat          float64       
 6   Lon          float64       
 7   Temperature  float64       
 8   RH_ice       float64       
 9   IsISSR       int64         
dtypes: datetime64[ns](1), float64(4), int64(5)
memory usage: 4.7 GB


### 6. Merge df with Air Traffic Control Center Values (ARTCC)

In [39]:
centers = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv')
centers

,IDENT,NAME,Nx,Ny
0,ZMA,MIAMI,233,28
1,ZMA,MIAMI,242,29
2,ZMA,MIAMI,232,28
3,ZMA,MIAMI,241,29
4,ZMA,MIAMI,249,30
...,...,...,...,...
25318,ZSE,SEATTLE,76,173
25319,ZLC,SALT LAKE CITY,114,168
25320,ZLC,SALT LAKE CITY,126,167
25321,ZMP,MINNEAPOLIS,144,166


In [40]:
%%time
df_centers = df.merge(centers, on= ["Nx", "Ny"])
df_centers

CPU times: user 11.1 s, sys: 2.04 s, total: 13.2 s
Wall time: 13.2 s


,dateTime,hPa,FLevel,Nx,Ny,Lat,Lon,Temperature,RH_ice,IsISSR,IDENT,NAME
0,2021-05-28 00:00:00,150,440,224,28,24.166930,-83.170346,209.040,22.21,0,ZMA,MIAMI
1,2021-05-28 00:00:00,175,410,224,28,24.166930,-83.170346,214.469,17.98,0,ZMA,MIAMI
2,2021-05-28 00:00:00,200,390,224,28,24.166930,-83.170346,220.164,13.07,0,ZMA,MIAMI
3,2021-05-28 00:00:00,225,360,224,28,24.166930,-83.170346,225.205,12.54,0,ZMA,MIAMI
4,2021-05-28 00:00:00,250,340,224,28,24.166930,-83.170346,229.820,9.42,0,ZMA,MIAMI
...,...,...,...,...,...,...,...,...,...,...,...,...
23702323,2021-05-30 23:00:00,350,270,54,177,48.968049,-123.224673,240.208,27.36,0,ZSE,SEATTLE
23702324,2021-05-30 23:00:00,375,250,54,177,48.968049,-123.224673,244.353,19.71,0,ZSE,SEATTLE
23702325,2021-05-30 23:00:00,400,240,54,177,48.968049,-123.224673,248.566,19.52,0,ZSE,SEATTLE
23702326,2021-05-30 23:00:00,425,220,54,177,48.968049,-123.224673,252.647,17.67,0,ZSE,SEATTLE


In [41]:
df_centers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23702328 entries, 0 to 23702327
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   dateTime     datetime64[ns]
 1   hPa          int64         
 2   FLevel       int64         
 3   Nx           int64         
 4   Ny           int64         
 5   Lat          float64       
 6   Lon          float64       
 7   Temperature  float64       
 8   RH_ice       float64       
 9   IsISSR       int64         
 10  IDENT        object        
 11  NAME         object        
dtypes: datetime64[ns](1), float64(4), int64(5), object(2)
memory usage: 2.3+ GB


### 6. Data Preprocess

https://github.com/vinsci/geohash

In [42]:
%%time
df_centers['geoEncode'] = [pgh.encode(df_centers['Lat'][i], df_centers['Lon'][i]) for i in range(len(df_centers))]

CPU times: user 24min 36s, sys: 0 ns, total: 24min 36s
Wall time: 24min 36s


In [43]:
df_centers

,dateTime,hPa,FLevel,Nx,Ny,Lat,Lon,Temperature,RH_ice,IsISSR,IDENT,NAME,geoEncode
0,2021-05-28 00:00:00,150,440,224,28,24.166930,-83.170346,209.040,22.21,0,ZMA,MIAMI,dhkc7rmnw6qg
1,2021-05-28 00:00:00,175,410,224,28,24.166930,-83.170346,214.469,17.98,0,ZMA,MIAMI,dhkc7rmnw6qg
2,2021-05-28 00:00:00,200,390,224,28,24.166930,-83.170346,220.164,13.07,0,ZMA,MIAMI,dhkc7rmnw6qg
3,2021-05-28 00:00:00,225,360,224,28,24.166930,-83.170346,225.205,12.54,0,ZMA,MIAMI,dhkc7rmnw6qg
4,2021-05-28 00:00:00,250,340,224,28,24.166930,-83.170346,229.820,9.42,0,ZMA,MIAMI,dhkc7rmnw6qg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23702323,2021-05-30 23:00:00,350,270,54,177,48.968049,-123.224673,240.208,27.36,0,ZSE,SEATTLE,c28qefqe9402
23702324,2021-05-30 23:00:00,375,250,54,177,48.968049,-123.224673,244.353,19.71,0,ZSE,SEATTLE,c28qefqe9402
23702325,2021-05-30 23:00:00,400,240,54,177,48.968049,-123.224673,248.566,19.52,0,ZSE,SEATTLE,c28qefqe9402
23702326,2021-05-30 23:00:00,425,220,54,177,48.968049,-123.224673,252.647,17.67,0,ZSE,SEATTLE,c28qefqe9402


In [44]:
df_centers['NAME'].value_counts()

SALT LAKE CITY    2749968
MINNEAPOLIS       2559024
SEATTLE           1755936
DENVER            1690416
ALBUQUERQUE       1424592
HOUSTON           1330056
JACKSONVILLE      1183104
OAKLAND           1126008
KANSAS CITY       1090440
LOS ANGELES       1082952
BOSTON            1001520
FORT WORTH         971568
MEMPHIS            903240
NEW YORK           872352
MIAMI              768456
WASHINGTON         745056
ATLANTA            651456
CHICAGO            645840
INDIANAPOLIS       588744
CLEVELAND          561600
Name: NAME, dtype: int64

In [45]:
df_centers.groupby(['FLevel'])['IsISSR'].aggregate('sum')

FLevel
210        0
220        0
240        0
250       26
270        5
280      791
300    21782
320    69485
340    19422
360    43227
390    23509
410      940
440        0
Name: IsISSR, dtype: int64

In [46]:
df_centers_select = df_centers.loc[df_centers['dateTime'] > pd.Timestamp(year= 2021, month= 5, day= 29, hour= 12)].copy()
df_centers_select

,dateTime,hPa,FLevel,Nx,Ny,Lat,Lon,Temperature,RH_ice,IsISSR,IDENT,NAME,geoEncode
481,2021-05-29 13:00:00,150,440,224,28,24.166930,-83.170346,206.586,36.21,0,ZMA,MIAMI,dhkc7rmnw6qg
482,2021-05-29 13:00:00,175,410,224,28,24.166930,-83.170346,212.201,34.97,0,ZMA,MIAMI,dhkc7rmnw6qg
483,2021-05-29 13:00:00,200,390,224,28,24.166930,-83.170346,218.829,31.76,0,ZMA,MIAMI,dhkc7rmnw6qg
484,2021-05-29 13:00:00,225,360,224,28,24.166930,-83.170346,224.908,20.50,0,ZMA,MIAMI,dhkc7rmnw6qg
485,2021-05-29 13:00:00,250,340,224,28,24.166930,-83.170346,229.834,18.82,0,ZMA,MIAMI,dhkc7rmnw6qg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23702323,2021-05-30 23:00:00,350,270,54,177,48.968049,-123.224673,240.208,27.36,0,ZSE,SEATTLE,c28qefqe9402
23702324,2021-05-30 23:00:00,375,250,54,177,48.968049,-123.224673,244.353,19.71,0,ZSE,SEATTLE,c28qefqe9402
23702325,2021-05-30 23:00:00,400,240,54,177,48.968049,-123.224673,248.566,19.52,0,ZSE,SEATTLE,c28qefqe9402
23702326,2021-05-30 23:00:00,425,220,54,177,48.968049,-123.224673,252.647,17.67,0,ZSE,SEATTLE,c28qefqe9402


In [47]:
takeALook = pd.DataFrame(df_centers.groupby(['hPa', 'NAME'])['IsISSR'].aggregate('sum'))
takeALook.reset_index(inplace= True)
takeALook.rename(columns= {'IsISSR': 'ISSRcounts'}, inplace= True)
takeALook.sort_values(by= ['ISSRcounts'], axis= 0, ascending= False)

,hPa,NAME,ISSRcounts
134,300,MINNEAPOLIS,10196
110,275,KANSAS CITY,9811
114,275,MINNEAPOLIS,9711
102,275,BOSTON,8900
105,275,DENVER,6960
...,...,...,...
155,325,NEW YORK,0
156,325,OAKLAND,0
159,325,WASHINGTON,0
160,350,ALBUQUERQUE,0


### 7. Organize Data for ML

### 8. Information on NOAA Pressure Level (hPa), Computed FL and Computed Altitudes

In [48]:
df_centers['hPa'].value_counts

<bound method IndexOpsMixin.value_counts of 0           150
1           175
2           200
3           225
4           250
           ... 
23702323    350
23702324    375
23702325    400
23702326    425
23702327    450
Name: hPa, Length: 23702328, dtype: int64>

Sources:

https://www.weather.gov/media/epz/wxcalc/pressureAltitude.pdf

In [49]:
hpa = [mb for mb in range(150, 475+1, 25)]
alt_ft = [int((1- (mb/1013.25)**(0.190284))*145366.45) for mb in hpa]
fl_level = [int(np.round(alt/100, -1)) for alt in alt_ft]

pd.DataFrame({'Pressure (hPa)':hpa, 'Altitude (ft)':alt_ft, 'Flight Level':fl_level})

,Pressure (hPa),Altitude (ft),Flight Level
0,150,44301,440
1,175,41293,410
2,200,38615,390
3,225,36195,360
4,250,33984,340
5,275,31946,320
6,300,30052,300
7,325,28282,280
8,350,26620,270
9,375,25051,250


### 9. Store Output to "partly-cloudy-common-area" Bucket

In [50]:
# Examine the list of files are already in the output_bucket (i.e., "partly-cloudy-common-area")
conn = boto3.client('s3')
contents = conn.list_objects(Bucket= output_bucket, Prefix= subfolder)['Contents']
for f in contents:
    print(f['Key'])

AnnualTotalISSR_CellHours_df.csv
JuneFirst2020_24hr_issr.csv
June_Month_ISSR.csv
MonthlyTotalISSR_CellHours_df.csv
TotalISSR_CellHours_byFL.csv
cleanedData_for_ML.csv
dailyPlaneCountsInAllLevels.csv
dailyPlaneCountsInAllLevels_sel.csv
dailyPlaneCountsInCenters_comb.csv
dailyPlaneCountsInCenters_selcomb.csv
hourlyPlaneCountsInRAPCells.csv
hourly_issr_summary.csv
issrDailyByCenter_pd.csv
issrPeriodByCenter_pd.csv
issrPlanesHourly.csv
miami_6_hour_flat.parquet/_temporary/0/_temporary/
periodicPlaneCountsInRAPCells.csv
proof_of_concept/adbs-conus-rap-merge/
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_0_100000.csv
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_1000000_1100000.csv
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_100000_200000.csv
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_1100000_1200000.csv
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_1200000_1300000.csv
proof_of_concept/adbs-conus-rap-merge/adsb_conus_rap_1300000_1400000.csv
proof_of_co

In [51]:
output_bucket = 'partly-cloudy-common-area'
outputFileName = 'selectAfileName.csv' # <--------------------------------------------- Specify

wr.s3.to_csv(df_select, f"s3://{output_bucket}/{outputFileName}", index=False)